In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [89]:
import requests
import pandas as pd
pd.set_option('max_columns', None)
import pyarrow
import json
from pyspark.sql import SparkSession
import glob
import json
from psycopg2 import connect

In [139]:
df = pd.read_csv('/project/NameList300.csv')

In [140]:
df

,Unnamed: 0,Name,URL
0,0,Bonoo Indian Tapas,https://www.tripadvisor.com/Restaurant_Review-...
1,1,Hibox,https://www.tripadvisor.com/Restaurant_Review-...
2,2,Devine Restaurant Coffee Bar,https://www.tripadvisor.com/Restaurant_Review-...
3,3,Scarlett Green,https://www.tripadvisor.com/Restaurant_Review-...
4,4,Indian Room,https://www.tripadvisor.com/Restaurant_Review-...
...,...,...,...
295,295,The Foyer At Claridge's,https://www.tripadvisor.com/Restaurant_Review-...
296,296,Il Girasole,https://www.tripadvisor.com/Restaurant_Review-...
297,297,Specialist Cellars,https://www.tripadvisor.com/Restaurant_Review-...
298,298,The Bull Steak Expert,https://www.tripadvisor.com/Restaurant_Review-...


In [142]:
res_list = list(df['Name'])

In [143]:
list_rest2 = [i.replace("'", "").replace("’", "") for i in res_list]

list_rest2

['Bonoo Indian Tapas',
 'Hibox',
 'Devine Restaurant Coffee Bar',
 'Scarlett Green',
 'Indian Room',
 'Nora Cafe',
 'Bayleaf Restaurant',
 'Amrutha Lounge',
 'Alexander The Great',
 'Andys Greek Taverna',
 'Muhib Indian Cuisine',
 'Chamisse',
 'Spasso',
 'Indian Moment',
 'Taste Of Nawab',
 'Awesome Thai #Finchley',
 'Bruce Burgers',
 'Tanias of Hampstead',
 'The Park Room',
 'The Wine Place Covent Garden',
 'Cafe Parisienne',
 'Manuels Restaurant and Bar',
 'The Rajdoot',
 'The Rajdoot Hampstead',
 'Launceston Place',
 'Room 43 Bar & Kitchen',
 'Paladar',
 'Dobar',
 'Eastern Eye Balti House',
 'O Gourmet Libanais',
 'Latitude Wimbledon',
 'Pivaz Hackney',
 'Lizzies Cucina - Italian Restaurant',
 'Fugitive Motel',
 'The Prince of Wales',
 'Yum Sa',
 'Wild Thyme',
 'Cafe de Provence',
 'Bebs Kitchen',
 'Almas Restaurant',
 'The Hampshire',
 'Brick Lane Brasserie',
 'Iznik',
 'The Lounge Cafe',
 'Cafe Amisha',
 '28-50 Wine Workshop & Kitchen South Kensington',
 'Core by Clare Smyth',
 'L

In [60]:
# function to get restaurant information using api and save into json
def get_data(restaurant):
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={restaurant}%20London&inputtype=textquery&fields=place_id&key=apikey"
    
    payload={}
    headers = {}
    response = requests.get(url, headers=headers, data=payload)

    restaurant_id = json.loads(response.text)
    rest = restaurant_id['candidates'][0]['place_id']
    
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={rest}&key=apikey"
    
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    
    json_data = response.json()
    
    google_id = {'google_id' : rest}
    
    json_data.update(google_id)
    
    res_name = json_data['result']['name']
    
    with open(f"/project/json/{res_name}.json", "w+") as json_file:
        json_string = json.dumps(json_data, sort_keys=True, indent=4)
        json_file.write(json_string)
    print('request completed')

In [145]:
for i in list_rest2:
    try:
        get_data(i)
    except:
        pass

request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request completed
request co

In [146]:
pathlist = []

In [147]:
for filepath in glob.iglob('/project/json/*.json'):
    pathlist.append(filepath)

In [148]:
pathlist

['/project/json/Fishcotheque.json',
 '/project/json/The Pride of Spitalfields London.json',
 '/project/json/The Clink Brixton.json',
 '/project/json/Muhib Indian cuisine.json',
 '/project/json/28-50 Wine Workshop & Kitchen.json',
 '/project/json/Dobar.json',
 '/project/json/Featherblade Steak Restaurant.json',
 '/project/json/Bel Canto.json',
 '/project/json/Bengal Village - Best of Brick Lane.json',
 '/project/json/The Ledbury.json',
 '/project/json/Azou Restaurant.json',
 '/project/json/The Great Chase.json',
 '/project/json/Bar Remo.json',
 '/project/json/Wolkite restaurant.json',
 "/project/json/Sam's Riverside.json",
 '/project/json/Luna Rossa.json',
 '/project/json/Peninsula Restaurant.json',
 '/project/json/Kahani Restaurant Sloane Square.json',
 '/project/json/EaTurkish.json',
 '/project/json/Indian Room.json',
 '/project/json/Lord Wargrave.json',
 '/project/json/Lorne.json',
 '/project/json/Core by Clare Smyth.json',
 '/project/json/The Bull Steak Expert.json',
 '/project/json

In [69]:
# create sparksession
spark = SparkSession \
    .builder \
    .appName("jsontoparquet") \
    .getOrCreate()

In [150]:
for i in pathlist:
    try:
        json_df = spark.read.json(i, multiLine=True)
        json_df.write.parquet(f"/project/parquet/{i[14:-5]}.parquet")
    except:
        pass

In [151]:
parquet_filepath = []

In [152]:
for filepath in glob.iglob('/project/parquet/*.parquet'):
    parquet_filepath.append(filepath)

In [153]:
parquet_filepath

['/project/parquet/Champagne Brunch at The Landmark London.parquet',
 '/project/parquet/Zeret Kitchen.parquet',
 '/project/parquet/Shahi Pakwaan.parquet',
 '/project/parquet/Ciullosteria.parquet',
 '/project/parquet/Buenos Aires.parquet',
 '/project/parquet/Nonna Selena Pizzeria.parquet',
 '/project/parquet/The India - Best of The City.parquet',
 '/project/parquet/TOKii.parquet',
 '/project/parquet/Trattoria Raffaele.parquet',
 '/project/parquet/Figo Leyton.parquet',
 '/project/parquet/Amber Restaurant.parquet',
 '/project/parquet/Chez Abir Lebanese Restaurant.parquet',
 '/project/parquet/The Bull Steak Expert.parquet',
 '/project/parquet/Core by Clare Smyth.parquet',
 '/project/parquet/Lorne.parquet',
 '/project/parquet/EaTurkish.parquet',
 '/project/parquet/Indian Room.parquet',
 '/project/parquet/Lord Wargrave.parquet',
 '/project/parquet/The Ledbury.parquet',
 '/project/parquet/Bel Canto.parquet',
 '/project/parquet/Bengal Village - Best of Brick Lane.parquet',
 '/project/parquet/D

In [154]:
def get_variables(data):
    google_id = data['google_id']
    identifier = data['result']['name'].replace(" ", "").replace("'", "").replace("’", "").lower()
    restaurant_name = data['result']['name'].replace("'", "").replace("’", "")
    try:
        address = data['result']['formatted_address'].replace("'", "")
    except:
        address = 'no address'
    try:
        postcode = data['result']['address_components'][6]['long_name']
    except:
        postcode = 'no postcode'
    try:
        website = data['result']['website']
    except:
        website = 'no website'
    try:
        phone_number = data['result']['formatted_phone_number']
    except:
        phone_number = 'no phone number'
    try:
        opening_hours = str(data['result']['opening_hours']['weekday_text'])
    except:
        opening_hours = 'no opening hours'
    try:
        rating = data['result']['rating']
    except:
        rating = 'no rating'
    try:
        total_ratings = data['result']['user_ratings_total']
    except:
        total_ratings = 'no total ratings'
    try:
        price_level = data['result']['price_level']
    except:
        price_level = 'no price level'
    
    
    return google_id, identifier,restaurant_name, address, postcode, website, phone_number, rating, total_ratings, price_level

In [155]:
def etl(restaurant):
    df = spark.read.parquet(restaurant)
    json_data = json.loads(df.toJSON().first())
    
    table_name = 'public.google'
    sql_string = f'INSERT INTO {table_name}\nVALUES {get_variables(json_data)};'

    conn = connect(
        dbname = "initialdatabase",
        user = "yourname",
        host = "demsin0166.czfwea5noxbs.eu-west-2.rds.amazonaws.com",
        password = "password")
    
    cur = conn.cursor()
    
    try:
        cur.execute( sql_string )
        conn.commit()
        print(f'{restaurant} added to database')
    except:
        print(f'{restaurant} already in database')
    
    cur.close()
    conn.close()
    

In [159]:
for i in parquet_filepath:
    etl(i)

/project/parquet/Champagne Brunch at The Landmark London.parquet already in database
/project/parquet/Zeret Kitchen.parquet added to database
/project/parquet/Shahi Pakwaan.parquet added to database
/project/parquet/Ciullosteria.parquet added to database
/project/parquet/Buenos Aires.parquet already in database
/project/parquet/Nonna Selena Pizzeria.parquet added to database
/project/parquet/The India - Best of The City.parquet added to database
/project/parquet/TOKii.parquet added to database
/project/parquet/Trattoria Raffaele.parquet added to database
/project/parquet/Figo Leyton.parquet added to database
/project/parquet/Amber Restaurant.parquet added to database
/project/parquet/Chez Abir Lebanese Restaurant.parquet already in database
/project/parquet/The Bull Steak Expert.parquet added to database
/project/parquet/Core by Clare Smyth.parquet added to database
/project/parquet/Lorne.parquet already in database
/project/parquet/EaTurkish.parquet added to database
/project/parquet/

/project/parquet/Tom Simmons - Tower Bridge.parquet added to database
/project/parquet/AW Restaurant.parquet added to database
/project/parquet/The Petite Corée.parquet already in database
/project/parquet/Alexander The Great Restaurant.parquet already in database
/project/parquet/Launceston Place.parquet already in database
/project/parquet/Anglo.parquet added to database
/project/parquet/Manuel's Italian and Mediterranean Restaurant and Bar.parquet already in database
/project/parquet/Moira Restaurant.parquet added to database
/project/parquet/Sushi Tetsu.parquet added to database
/project/parquet/La Trompette.parquet added to database
/project/parquet/Andy’s Greek Taverna.parquet already in database
/project/parquet/Café Amisha.parquet already in database
/project/parquet/Volare.parquet added to database
/project/parquet/Baileys.parquet already in database
/project/parquet/Wiltons Restaurant.parquet already in database
/project/parquet/Mosob Restaurant.parquet added to database
/pro

/project/parquet/Barge East Restaurant.parquet added to database
/project/parquet/Ormer Mayfair Restaurant.parquet already in database
/project/parquet/Almas Restaurant.parquet added to database
/project/parquet/Leicester Square Kitchen.parquet added to database
/project/parquet/Becks cafe.parquet added to database
/project/parquet/The Hayden Pub & Rooms.parquet already in database
/project/parquet/Bella Roma - Italian Restaurant.parquet added to database
/project/parquet/Afternoon Tea at The Park Room.parquet already in database
/project/parquet/Mangia Bene.parquet added to database
/project/parquet/Dulce Coffee London.parquet already in database
/project/parquet/Taste of Nawab.parquet already in database
/project/parquet/The Chelsea Corner.parquet added to database
/project/parquet/NORA CAFE.parquet already in database
/project/parquet/Blacklock City.parquet added to database
/project/parquet/May Fair Kitchen.parquet added to database
/project/parquet/Blacklock Soho.parquet added to 